In [16]:
!pip install -q ja_ginza_electra pandarallel

In [18]:
# 前処理用
import re
from tqdm.auto import tqdm
tqdm.pandas()
import os
import pandas as pd
import numpy as np
import emoji
import spacy
import re
import neologdn
import json

# 分かち書き用
import ginza
import ja_ginza_electra
from pandarallel import pandarallel # ライブラリー
pandarallel.initialize(progress_bar=True)

train = pd.read_csv("../data/train.csv").head(100)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [19]:
def wakati_rm_func(x):
    nlp = spacy.load('ja_ginza_electra')
    sentence = x
    
    sentence = re.sub(r'[!-~]'," ",sentence,) # 小文字の記号を削除
    sentence=re.sub(r'[︰-＠]', "", sentence) # 大文字の記号を削除
    # 絵文字削除
    sentence = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in sentence])

    # 不要記号削除
    pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％�]'
    sentence =  re.sub(pattern, ' ', sentence)

    # 正規化する
    sentence = neologdn.normalize(sentence)
 

    # 大文字・小文字変換
    sentence = sentence.lower()

    # GinZaで分かち書きをする
    doc = nlp(sentence)
    nlp = spacy.load('ja_ginza_electra')
    tmp_words_list = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ in ["PROPN","NOUN", "ADJ", "VERB"]:
                tmp_words_list.append(token.orth_)

    result = " ".join(tmp_words_list)
    return result

In [20]:
train["story"] = train["story"].progress_apply(wakati_rm_func)

  0%|          | 0/100 [00:00<?, ?it/s]

Cannot find the requested files in the cached path and outgoing traffic has been disabled. To enable model look-ups and downloads online, set 'local_files_only' to False.


Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/434M [00:00<?, ?B/s]

In [ ]:
train